In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
#Importing some libraries

In [2]:

from keras.preprocessing.image import ImageDataGenerator

# Set up the ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)  # Normalize pixel values between 0 and 1 of R, G and B
batch_size = 32  # Adjust batch size as per your requirements
training_data = './train/' #Loading the training data directory
train_ds = train_datagen.flow_from_directory( #normalised image. flow from directory
       training_data, #Directory
       target_size=(128,128), #Image size
       batch_size=batch_size, #batch size
       class_mode='binary'
)


Found 557 images belonging to 2 classes.


In [3]:
testing_data = './test/'
test_datagen = ImageDataGenerator(rescale=1./255)
val_ds = test_datagen.flow_from_directory(
  testing_data,
  target_size=(128, 128),
  batch_size=batch_size,
  class_mode='binary'
)

Found 140 images belonging to 2 classes.


In [4]:
from keras import Sequential
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.losses import BinaryCrossentropy
model = Sequential([
                    Conv2D(16,(3,3),activation='relu',input_shape=(128,128,3)), #16 Filters, 3x3 Filters. A filter identifies the pattern from the input, so in the first layer it will check for horizontal lines, vertical lines.etc and so I have put only 16 filters.
                    MaxPool2D(2,2), #pooling compresses the image, (2,2) means it will take 4 pixels and return the darkest one
                    Conv2D(32,(3,3),activation='relu'),
                    MaxPool2D(2,2),
                    Flatten(), #Flatten turns the 2D dimensions of an image into a 1D array for linear computation
                    Dense(512,activation='relu'),
                    Dense(1,activation='sigmoid')    #Sigmoid function for binary classification
                    ])
model.compile(optimizer='adam',loss=BinaryCrossentropy(),metrics=['accuracy']) #Adam is an optimiser like gradient descent, but it can adjust alpha automatically

In [5]:
history = model.fit(train_ds,validation_data=val_ds,epochs=15) #fitting the model and training it
            

Epoch 1/15
18/18 [==============================] - 8s 401ms/step - loss: 1.6587 - accuracy: 0.5117 - val_loss: 0.7069 - val_accuracy: 0.5000
Epoch 2/15
18/18 [==============================] - 7s 367ms/step - loss: 0.6850 - accuracy: 0.6014 - val_loss: 0.6763 - val_accuracy: 0.5500
Epoch 3/15
18/18 [==============================] - 7s 366ms/step - loss: 0.5935 - accuracy: 0.7020 - val_loss: 0.6669 - val_accuracy: 0.6357
Epoch 4/15
18/18 [==============================] - 6s 348ms/step - loss: 0.4961 - accuracy: 0.7899 - val_loss: 0.6591 - val_accuracy: 0.5857
Epoch 5/15
18/18 [==============================] - 6s 339ms/step - loss: 0.3977 - accuracy: 0.8438 - val_loss: 0.6115 - val_accuracy: 0.6429
Epoch 6/15
18/18 [==============================] - 6s 342ms/step - loss: 0.2859 - accuracy: 0.9300 - val_loss: 0.6300 - val_accuracy: 0.6571
Epoch 7/15
18/18 [==============================] - 6s 349ms/step - loss: 0.2155 - accuracy: 0.9336 - val_loss: 0.7724 - val_accuracy: 0.6143
Epoch 

In [8]:
model.save('./Saved Models/mymodel') #saving the model to a directory so we dont have to run the training every time

INFO:tensorflow:Assets written to: ./Saved Models/mymodel\assets


INFO:tensorflow:Assets written to: ./Saved Models/mymodel\assets


In [1]:
from keras.models import load_model
model = load_model('./Saved Models/mymodel') #loading the model

In [6]:
import cv2 #the library used for image processing
import numpy as np
image = cv2.imread('./bear.jpg')
resized_image = cv2.resize(image, (128, 128))  # Resize to match the input shape of your model
normalized_image = resized_image / 255.0  # Normalize pixel values to the range of [0, 1]
preprocessed_image = np.expand_dims(normalized_image, axis=0)  # Add an extra dimension for batch size (assuming single image) - idk

# Perform inference with your model
predictions = model.predict(preprocessed_image)
if predictions<0.5:
       print("Cat")
else:
       print("Dog")

1/1 [==============================] - 0s 23ms/step
Cat
